Data exploration

In [2]:
import pandas as pd

news_df = pd.read_csv("Sora_LREC2020_biasedsentences.csv")

Seems that each instance consists of an article with sentences and a bias score between 1 and 4 is assigned to the article as a whole, the title, and each sentence.

In [28]:
news_test_sentence = news_df.iloc[0] # visual inspection
print(f"sentence: {news_test_sentence['s0']}", f"score: {news_test_sentence["0"]}")

print(f"dataset score range: {news_df["article_bias"].min(),news_df["article_bias"].max()}")

sentence: [0]: LOUISVILLE - Dan Johnson posted a final message on Facebook to his friends and family on Wednesday afternoon. score: 3
dataset score range: (np.int64(1), np.int64(4))


Baseline

In [ ]:
# source: https://huggingface.co/d4data/bias-detection-model
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer) # cuda = 0,1 based on gpu availability

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at d4data/bias-detection-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [43]:
def rescale(score):
    '''baseline classifier gives scores in the range of 0.5 to 1 (i think)
        making a linear function to map the score to the correct range
    '''
    current_min_score = 1
    current_max_score = 4
    target_min_score = 0.5
    target_max_score = 1
    m = (target_max_score-target_min_score)/(current_max_score-current_min_score)
    b = target_min_score-(m*current_min_score)
    
    return (m*score)+b

# comparing bias score obtained from classifier to true bias score for a test sentence from the news dataset
test_sent_pred = classifier(news_test_sentence["s0"])
test_sent_y = rescale(news_test_sentence["0"])
print(f"pred: {test_pred[-1]['score']}, y: {test_sent_y}")

pred: 0.762514591217041, y: 0.8333333333333334


In [ ]:
# compute mean squared error for all individual sentences and titles in the news dataset

# iterate through every instance
# iterate through every sentence (and title)
# compute bias score with classifier
# compute square error as the square of the difference between the predicted and label
# end
# end
# divide the total square erorr by the total number of sentences (keep track with counter) to obtain MSE

Fine tune classifier and compare to baseline

In [ ]:
...

Make predictions for X notes data

In [ ]:
...

Compare classifier predictions to human predictions

In [ ]:
# my idea for how to evaluate:
# 1) we set some guidelines for how we will evaluate notes 
# 2) we individually make labels for one days worth of notes (that's about 200 notes; this would be the train set)
# 3) we calculate out interrater agreement (if it's very bad, we revise our guidelines and re-label)
# 4) we average our labels and use that to further fine-tune the classifier
# 5) we repeat step 2 on a new set of notes (this would be the test set)
# 6) we make predictions using the classifier obtained from step 4
# 7) either we calculate the MSE using our labels as ground truth, or we calculate three interrater agreements (chico-andrew, chico-classifier, andrew-classifier)
# and see if the human-human agreement is better than the human-classifier agreement.
...